In [18]:
!pip install chromadb
!pip install langchain chromadb transformers sentence-transformers
!pip install langchain chromadb pymupdf
!pip install -U langchain-community

In [19]:
# Импорт библиотек
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import json
import torch

In [20]:
bank_knowledge = [
    {
        "income_level": "низкий",
        "purpose": "потребительский",
        "amount": "малый",
        "recommendation": {
            "bank": "Тинькофф",
            "product": "Потребительский кредит",
            "rate": "15%",
            "advantages": "Быстрое одобрение без залога",
            "link": "https://www.tinkoff.ru/loans/"
        }
    },
    {
        "income_level": "средний",
        "purpose": "ипотека",
        "amount": "средний",
        "recommendation": {
            "bank": "Сбербанк",
            "product": "Ипотека",
            "rate": "12%",
            "advantages": "Господдержка для семей с детьми",
            "link": "https://www.sberbank.ru/ru/homeloans"
        }
    },
    {
        "income_level": "высокий",
        "purpose": "бизнес",
        "amount": "крупный",
        "recommendation": {
            "bank": "Альфа-Банк",
            "product": "Бизнес-кредит",
            "rate": "10%",
            "advantages": "Индивидуальные условия для ИП",
            "link": "https://alfabank.ru/sme/business/loans/"
        }
    }
]

In [21]:
with open('bank_knowledge.json', 'w', encoding='utf-8') as f:
    json.dump(bank_knowledge, f, ensure_ascii=False, indent=4)

In [22]:
model_name = "/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2"  # Бесплатная модель на Kaggle

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [24]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

Device set to use cuda:0


In [25]:
llm = HuggingFacePipeline(pipeline=pipe)

In [26]:
def load_knowledge():
    with open('bank_knowledge.json', 'r', encoding='utf-8') as f:
        return json.load(f)

In [27]:
def format_knowledge(knowledge):
    rules = []
    for item in knowledge:
        rule = (
            f"Если: Доход = {item['income_level']}, "
            f"Цель = {item['purpose']}, "
            f"Сумма = {item['amount']}\n"
            f"То: Банк = {item['recommendation']['bank']}, "
            f"Продукт = {item['recommendation']['product']}, "
            f"Ставка = {item['recommendation']['rate']}, "
            f"Преимущества = {item['recommendation']['advantages']}, "
            f"Ссылка = {item['recommendation']['link']}"
        )
        rules.append(rule)
    return "\n\n".join(rules)

In [28]:
system_prompt_template = """
Ты - финансовый эксперт. Используй следующие правила для рекомендаций:

{knowledge_base}

Дополнительные инструкции:
1. Отвечай только на русском языке
2. Будь вежливым и профессиональным
3. Если нет точного совпадения по правилам, предложи ближайшие варианты
4. Всегда указывай ссылку на официальный сайт банка
5. Объясняй преимущества каждого предложения

Запрос клиента: {user_input}

Твой развернутый ответ:
"""

In [29]:
prompt = PromptTemplate(
    input_variables=["knowledge_base", "user_input"],
    template=system_prompt_template
)

In [30]:
recommendation_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False
)

In [31]:
def get_user_preferences():
    print("\n" + "="*40)
    print("ФИНАНСОВЫЙ КОНСУЛЬТАНТ".center(40))
    print("="*40)
    
    print("\nПожалуйста, введите ваши данные:")
    income = input("Уровень дохода (низкий/средний/высокий): ").strip().lower()
    purpose = input("Цель кредита: ").strip().lower()
    amount = input("Сумма кредита (малый/средний/крупный): ").strip().lower()
    
    return {
        "income": income,
        "purpose": purpose,
        "amount": amount
    }

In [32]:
def format_user_input(preferences):
    return (
        f"Доход: {preferences['income']}, "
        f"Цель: {preferences['purpose']}, "
        f"Сумма: {preferences['amount']}"
    )

In [33]:
def main():
    print("\nДобро пожаловать в систему подбора кредитов!")
    print("Мы подберем для вас оптимальное предложение.\n")
    
    # Загрузка базы знаний
    knowledge = load_knowledge()
    formatted_knowledge = format_knowledge(knowledge)
    
    while True:
        try:
            # Получение ввода пользователя
            preferences = get_user_preferences()
            user_input = format_user_input(preferences)
            
            # Генерация рекомендации
            recommendation = recommendation_chain.run(
                knowledge_base=formatted_knowledge,
                user_input=user_input
            )
            
            # Вывод результата
            print("\n" + "="*40)
            print("РЕКОМЕНДАЦИЯ".center(40))
            print("="*40)
            print(recommendation.strip())
            
            # Проверка на продолжение
            if input("\nХотите сделать новый запрос? (да/нет): ").lower().strip() != 'да':
                print("\nСпасибо за использование нашего сервиса!")
                break
                
        except KeyboardInterrupt:
            print("\nРабота программы завершена.")
            break
        except Exception as e:
            print(f"\nПроизошла ошибка: {str(e)}")
            print("Попробуйте еще раз.")

In [34]:
if __name__ == "__main__":
    # Очистка памяти GPU перед запуском
    torch.cuda.empty_cache()
    main()


Добро пожаловать в систему подбора кредитов!
Мы подберем для вас оптимальное предложение.


         ФИНАНСОВЫЙ КОНСУЛЬТАНТ         

Пожалуйста, введите ваши данные:


Уровень дохода (низкий/средний/высокий):  низкий
Цель кредита:  потребительский
Сумма кредита (малый/средний/крупный):  малый



              РЕКОМЕНДАЦИЯ              
Ты - финансовый эксперт. Используй следующие правила для рекомендаций:

Если: Доход = низкий, Цель = потребительский, Сумма = малый
То: Банк = Тинькофф, Продукт = Потребительский кредит, Ставка = 15%, Преимущества = Быстрое одобрение без залога, Ссылка = https://www.tinkoff.ru/loans/

Если: Доход = средний, Цель = ипотека, Сумма = средний
То: Банк = Сбербанк, Продукт = Ипотека, Ставка = 12%, Преимущества = Господдержка для семей с детьми, Ссылка = https://www.sberbank.ru/ru/homeloans

Если: Доход = высокий, Цель = бизнес, Сумма = крупный
То: Банк = Альфа-Банк, Продукт = Бизнес-кредит, Ставка = 10%, Преимущества = Индивидуальные условия для ИП, Ссылка = https://alfabank.ru/sme/business/loans/

Дополнительные инструкции:
1. Отвечай только на русском языке
2. Будь вежливым и профессиональным
3. Если нет точного совпадения по правилам, предложи ближайшие варианты
4. Всегда указывай ссылку на официальный сайт банка
5. Объясняй преимущества каждого п


Хотите сделать новый запрос? (да/нет):  да



         ФИНАНСОВЫЙ КОНСУЛЬТАНТ         

Пожалуйста, введите ваши данные:


Уровень дохода (низкий/средний/высокий):  средний
Цель кредита:  Ипотека
Сумма кредита (малый/средний/крупный):  среднийй



              РЕКОМЕНДАЦИЯ              
Ты - финансовый эксперт. Используй следующие правила для рекомендаций:

Если: Доход = низкий, Цель = потребительский, Сумма = малый
То: Банк = Тинькофф, Продукт = Потребительский кредит, Ставка = 15%, Преимущества = Быстрое одобрение без залога, Ссылка = https://www.tinkoff.ru/loans/

Если: Доход = средний, Цель = ипотека, Сумма = средний
То: Банк = Сбербанк, Продукт = Ипотека, Ставка = 12%, Преимущества = Господдержка для семей с детьми, Ссылка = https://www.sberbank.ru/ru/homeloans

Если: Доход = высокий, Цель = бизнес, Сумма = крупный
То: Банк = Альфа-Банк, Продукт = Бизнес-кредит, Ставка = 10%, Преимущества = Индивидуальные условия для ИП, Ссылка = https://alfabank.ru/sme/business/loans/

Дополнительные инструкции:
1. Отвечай только на русском языке
2. Будь вежливым и профессиональным
3. Если нет точного совпадения по правилам, предложи ближайшие варианты
4. Всегда указывай ссылку на официальный сайт банка
5. Объясняй преимущества каждого п


Хотите сделать новый запрос? (да/нет):  нет



Спасибо за использование нашего сервиса!
